In [73]:
import pandas as pd
import random as rd
import numpy as np

In [67]:
def criarDF(nomeTxt):
    lista = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&', '–', '”', '“', '…', '=', ',', '.', '–', '"', '‘', '’', '"', ']', '’']
    txt = open(nomeTxt, 'r', encoding='utf-8')
    dfA = pd.DataFrame()
    linhas = txt.readlines()
    for linha in linhas:
        vetor = linha.replace("'", '').replace('[', '').replace(']\n', '').split(', ')
        for i in range(len(vetor)):
            for char in lista:
                vetor[i] = vetor[i].replace(char, '')

        dfA = pd.concat([dfA, pd.DataFrame(data=[[vetor]], columns=['noticias'])], ignore_index=True)
    return dfA

In [68]:
dfFalso = criarDF('../data/train/fakeTrain.txt')
dfReal = criarDF('../data/train/realTrain.txt')

In [69]:
def separarTreinoTeste(dfA):
    dfB = pd.DataFrame()
    dfC = pd.DataFrame()
    for i in range (len(dfA)):
        vetor = dfA.iloc[i, 0]
        if (rd.random() > 0.8): # vai p teste
            dfB = pd.concat([dfB, pd.DataFrame(data=[[vetor]], columns=['noticias'])], ignore_index=True)
        else:
            dfC = pd.concat([dfC, pd.DataFrame(data=[[vetor]], columns=['noticias'])], ignore_index=True)

    return dfB, dfC

In [70]:
def prepararDfTeste (dfA, dfB):
    dados = []

    # embaralha os dfs
    dfEmbaralhado = pd.concat([dfA, dfB], ignore_index=True)
    indices = list(dfEmbaralhado.index)
    rd.shuffle(indices)
    dfEmbaralhado = dfEmbaralhado.iloc[indices].reset_index(drop=True)
    
    # cria uma coluna pra tag
    for i in range (len(dfEmbaralhado)):
        news = dfEmbaralhado.iloc[i, 0]
        tag = news[0]
        news = news[1:]
        dados.append([tag, news])

    return pd.DataFrame(dados, columns=['tag', 'noticia'])

In [ ]:
def prepararDfsTreino (dfF, dfR):
    for i in range(len(dfF)):
        noticia = dfF.iloc[i, 0]
        noticia.remove('f')

    for i in range(len(dfR)):
        noticia = dfR.iloc[i, 0]
        noticia.remove('r')

    # Falta fazer a logica pra separar por palavras  

    return dfF, dfR

In [ ]:
def evaluate(dfReal, dfFalso, noticiasTeste):
    # Junta os dataframes e une as palavras
    dfJunto = dfReal.merge(dfFalso, how='outer', on='noticias')

    # Conta quantas palavras tem em uma e não tem na outra
    nReal = dfJunto.loc[dfJunto['aparicoesReais'].isna(), 'palavras'].count()
    nFalsa = dfJunto.loc[dfJunto['aparicoesFalsas'].isna(), 'palavras'].count()

    # Preenche as palavras que não aparecem com 1
    dfJunto.fillna(1, inplace=True)

    # Soma as palavras com 0 aparições com 1 para evitar multiplicação por 0
    dfJunto.loc[dfJunto['aparicoesReais'] > 1, 'aparicoesReais'] += 1
    dfJunto.loc[dfJunto['aparicoesFalsas'] > 1, 'aparicoesFalsas'] += 1

    # Soma o total de aparições
    totalReal = dfJunto['aparicoesReais'].sum()
    totalFalso = dfJunto['aparicoesFalsas'].sum()

    # Balanceia a ordem de grandeza das aparições
    dfJunto['aparicoesFalsas'] *= totalReal / totalFalso

    # Faz o coeficiente de bray courtis
    dfJunto['bc'] = abs(dfJunto['aparicoesReais'] - dfJunto['aparicoesFalsas']) / (dfJunto['aparicoesReais'] + dfJunto['aparicoesFalsas'])

    # Limpa palavras com coeficiente menor que 0.3 (palavras com aparições equivalentes em ambos casos)
    dfJunto = dfJunto.loc[dfJunto['bc'] > 0.3]

    # Pega o total de aparições após NLP
    tFalsas = dfJunto['aparicoesFalsas'].sum()
    tReais = dfJunto['aparicoesReais'].sum()

    # Define um df de teste
    dfJuntoTeste = dfJunto

    # Faz os calculos das probabilidades
    dfJuntoTeste['aparicoesFalsas'] = dfJunto['aparicoesFalsas'] / (nFalsa + tFalsas)
    dfJuntoTeste['aparicoesReais'] = dfJunto['aparicoesReais'] / (nReal + tReais)

    # Pega o log na base 10
    dfJuntoTeste['aparicoesFalsas'] = np.log10(dfJuntoTeste['aparicoesFalsas'])
    dfJuntoTeste['aparicoesReais'] = np.log10(dfJuntoTeste['aparicoesReais'])

    # NOTICIA:
    
    indices = ['fF', 'fR', 'rF', 'rR']
    dfDados = pd.DataFrame(0, index=indices, columns=['num'])

    for i in range (len(noticiasTeste)):
        noticia = noticiasTeste.iloc[i, 1]
        tag = noticiasTeste.iloc[i, 0]

        # Cria um df com a notícia limpa
        dfNoticia = pd.DataFrame(data = noticia, columns=['palavra'])
        print(dfNoticia)

        # Junta o df da noticia com o das probabilidades
        dfNoticia = dfNoticia.merge(dfJuntoTeste, how='inner', left_on='palavra', right_on='palavras')

        # Faz a soma das probabilidades
        iFalsa = dfNoticia['aparicoesFalsas'].sum()
        iReal = dfNoticia['aparicoesReais'].sum()

        # Confere o resultado
        if (iFalsa > iReal): # Falsa
            resultado = 'F'
        else:
            resultado = 'R'
            
        chave = tag + resultado
        dfDados.loc[chave, 'num'] += 1
        
    return dfDados

In [ ]:
for i in range(36):
    [dfFalsoTeste, dfFalsoTreino] = separarTreinoTeste(dfFalso)
    [dfRealTeste, dfRealTreino] = separarTreinoTeste(dfReal)

    dfTestes = prepararDfTeste(dfFalsoTeste, dfRealTeste)
    [dfFalsoTreino, dfRealTreino] = prepararDfsTreino(dfFalsoTreino, dfRealTreino)

    # data = evaluate(dfRealTreino, dfFalsoTreino, dfTestes)
    

CÓDIGO ANTIGO

In [ ]:
dfJunto = dfReal.merge(dfFalso, how='outer')

In [ ]:
nReal = dfJunto.loc[dfJunto['aparicoesReais'].isna(), 'palavras'].count()
nFalsa = dfJunto.loc[dfJunto['aparicoesFalsas'].isna(), 'palavras'].count()

In [ ]:
dfJunto.fillna(1, inplace=True)

In [ ]:
dfJunto.loc[dfJunto['aparicoesReais'] > 1, 'aparicoesReais'] += 1
dfJunto.loc[dfJunto['aparicoesFalsas'] > 1, 'aparicoesFalsas'] += 1

In [ ]:
totalReal = dfJunto['aparicoesReais'].sum()
totalFalso = dfJunto['aparicoesFalsas'].sum()

In [ ]:
dfJunto['aparicoesFalsas'] *= totalReal / totalFalso

In [ ]:
dfJunto['bc'] = abs(dfJunto['aparicoesReais'] - dfJunto['aparicoesFalsas']) / (dfJunto['aparicoesReais'] + dfJunto['aparicoesFalsas'])

In [ ]:
dfJunto = dfJunto.loc[dfJunto['bc'] > 0.3]

In [ ]:
tFalsas = dfJunto['aparicoesFalsas'].sum()
tReais = dfJunto['aparicoesReais'].sum()

In [ ]:
dfJunto

,palavras,aparicoesReais,aparicoesFalsas,bc
0,%,1.0,26.093942,0.926183
1,&,1.0,14.403856,0.870162
4,-pe,1.0,3.966279,0.597284
7,0,5.0,0.626255,0.777381
13,0000,1.0,3.966279,0.597284
...,...,...,...,...
19233,😡,1.0,11.272583,0.837035
19234,🙄🙄🙄,1.0,22.336414,0.914297
19235,🚨,1.0,3.966279,0.597284
19236,🤣🤣🤣,1.0,22.336414,0.914297


In [ ]:
dfJuntoTeste = dfJunto

In [ ]:
dfJuntoTeste['aparicoesFalsas'] = dfJunto['aparicoesFalsas'] / (nFalsa + tFalsas)
dfJuntoTeste['aparicoesReais'] = dfJunto['aparicoesReais'] / (nReal + tReais)
dfJuntoTeste

C:\Users\CEITELABINFO\AppData\Local\Temp\ipykernel_12024\229914731.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfJuntoTeste['aparicoesFalsas'] = dfJunto['aparicoesFalsas'] / (nFalsa + tFalsas)
C:\Users\CEITELABINFO\AppData\Local\Temp\ipykernel_12024\229914731.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfJuntoTeste['aparicoesReais'] = dfJunto['aparicoesReais'] / (nReal + tReais)


,palavras,aparicoesReais,aparicoesFalsas,bc
0,%,0.000003,0.000086,0.926183
1,&,0.000003,0.000047,0.870162
4,-pe,0.000003,0.000013,0.597284
7,0,0.000016,0.000002,0.777381
13,0000,0.000003,0.000013,0.597284
...,...,...,...,...
19233,😡,0.000003,0.000037,0.837035
19234,🙄🙄🙄,0.000003,0.000073,0.914297
19235,🚨,0.000003,0.000013,0.597284
19236,🤣🤣🤣,0.000003,0.000073,0.914297


In [ ]:
import numpy as np

In [ ]:
dfJuntoTeste['aparicoesFalsas'] = np.log10(dfJuntoTeste['aparicoesFalsas'])
dfJuntoTeste['aparicoesReais'] = np.log10(dfJuntoTeste['aparicoesReais'])

C:\Users\CEITELABINFO\AppData\Local\Temp\ipykernel_12024\2118644562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfJuntoTeste['aparicoesFalsas'] = np.log10(dfJuntoTeste['aparicoesFalsas'])
C:\Users\CEITELABINFO\AppData\Local\Temp\ipykernel_12024\2118644562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfJuntoTeste['aparicoesReais'] = np.log10(dfJuntoTeste['aparicoesReais'])


In [ ]:
noticia = 'Agora não sei se você viu lá no grupo que eu postei hoje, as pessoas novas que tiraram o título esse ano ou os que transferiram, que foi o meu caso, que eu transferi de Caldas Novas para cá, o título, eles dão um papel só, uma folha de papel compridinho assim, que é para você registrar na hora. Aí lá tem um QR Code que tem escrito Lula 13.'
noticia = noticia.lower()
noticia = noticia.split()
lista = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&']
noticiaFormatada = []

for palavras in noticia:
    for char in lista:
        palavras = palavras.replace(char, '')
    noticiaFormatada.append(palavras)

In [ ]:
dfNoticia = pd.DataFrame(data = noticiaFormatada, columns=['palavra'])
dfNoticia

,palavra
0,agora
1,não
2,sei
3,se
4,você
...,...
65,que
66,tem
67,escrito
68,lula


In [ ]:
dfNoticia = dfNoticia.merge(dfJuntoTeste, how='inner', left_on='palavra', right_on='palavras')

In [ ]:
iFalsa = dfNoticia['aparicoesFalsas'].sum()
iReal = dfNoticia['aparicoesReais'].sum()

In [ ]:
print(f'{iFalsa}\t{iReal}')

-98.09150096776183	-116.72186390297516


In [ ]:
if (iFalsa > iReal):
    print("Noticia Falsa!")
else:
    print("Noticia Real!")

Noticia Falsa!
